In [1]:
from veg2hab.criteria import FGRCriterium, LBKCriterium, BodemCriterium, EnCriteria, NietCriterium, OfCriteria
from veg2hab.enums import FGRType, LBKType, BodemType, LBKTuple
import pandas as pd
from enum import Enum

/home/jordydelange/.cache/pypoetry/virtualenvs/veg2hab-CuqoUkZb-py3.7/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.11.2-CAPI-1.17.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


FGR

In [2]:
du = FGRCriterium(wanted_fgrtype=FGRType.DU)

for fgr_type in [FGRType.DU, FGRType.HL, pd.NA]:
    row = pd.Series({"fgr": fgr_type})
    du.check(row)
    print(fgr_type)
    print(du.evaluation)
    print(du.get_opm())
    print("--------")

FGRType.DU
TRUE
{'FGR type is Duinen.'}
--------
FGRType.HL
FALSE
{'FGR type is niet Duinen, maar Heuvelland.'}
--------
<NA>
CANNOT_BE_AUTOMATED
{'Dit vlak ligt niet mooi binnen één FGR-vlak.'}
--------


Bodemkaart met enkel negatieven

In [3]:
lv = BodemCriterium(wanted_bodemtype=BodemType.LEEMARME_VAAGGRONDEN_H9190)

for bodem_code in [["Zn21"], ["Abcd"], [pd.NA]]:
    row = pd.Series({"bodem": bodem_code})
    lv.check(row)
    print(bodem_code)
    print(lv.evaluation)
    print(lv.get_opm())
    print("--------")

['Zn21']
CANNOT_BE_AUTOMATED
{'Dit is mogelijk Leemarme vaaggronden want bodemcode Zn21.'}
--------
['Abcd']
FALSE
{'Dit is niet Leemarme vaaggronden want bodemcode Abcd.'}
--------
[<NA>]
CANNOT_BE_AUTOMATED
{'Dit vlak ligt niet mooi binnen één bodemkaartvlak.'}
--------


Bodemkaart met zowel positieven als negatieven

In [4]:
vg = BodemCriterium(wanted_bodemtype=BodemType.VAAGGRONDEN)

for bodem_code in [["Zn21"], ["Abcd"], [pd.NA]]:
    row = pd.Series({"bodem": bodem_code})
    vg.check(row)
    print(bodem_code)
    print(vg.evaluation)
    print(vg.get_opm())
    print("--------")

['Zn21']
TRUE
{'Dit is Vaaggronden want bodemcode Zn21.'}
--------
['Abcd']
FALSE
{'Dit is niet Vaaggronden want bodemcode Abcd.'}
--------
[<NA>]
CANNOT_BE_AUTOMATED
{'Dit vlak ligt niet mooi binnen één bodemkaartvlak.'}
--------


LBK met enkel negatieven

In [5]:
hh = LBKCriterium(wanted_lbktype=LBKType.HERSTELLEND_HOOGVEEN)

for lbk_code in ["HzHL", "Abcd", pd.NA]:
    row = pd.Series({"lbk": lbk_code})
    hh.check(row)
    print(lbk_code)
    print(hh.evaluation)
    print(hh.get_opm())
    print("--------")

HzHL
CANNOT_BE_AUTOMATED
{'Dit is mogelijk Herstellend hoogveen want LBK code HzHL.'}
--------
Abcd
FALSE
{'Dit is niet Herstellend hoogveen want LBK code Abcd.'}
--------
<NA>
CANNOT_BE_AUTOMATED
{'Dit vlak ligt niet mooi binnen één LBK-vak'}
--------


LBK met enkel positieven

In [6]:
hl = LBKCriterium(wanted_lbktype=LBKType.HOOGVEENLANDSCHAP)

for lbk_code in ["HzHL", "Abcd", pd.NA]:
    row = pd.Series({"lbk": lbk_code})
    hl.check(row)
    print(lbk_code)
    print(hl.evaluation)
    print(hl.get_opm())
    print("--------")

HzHL
TRUE
{'Dit is Hoogveenlandschap want LBK code HzHL.'}
--------
Abcd
CANNOT_BE_AUTOMATED
{'Dit is mogelijk toch Hoogveenlandschap ondanks LBK code Abcd.'}
--------
<NA>
CANNOT_BE_AUTOMATED
{'Dit vlak ligt niet mooi binnen één LBK-vak'}
--------


Combineren

In [7]:
crit = EnCriteria(
    sub_criteria=
    [
        OfCriteria(
            sub_criteria=
            [
                LBKCriterium(wanted_lbktype=LBKType.HERSTELLEND_HOOGVEEN),
                LBKCriterium(wanted_lbktype=LBKType.HOOGVEENLANDSCHAP),
                BodemCriterium(wanted_bodemtype=BodemType.LEEMARME_HUMUSPODZOLGRONDEN),
            ]), 
        NietCriterium(
            sub_criterium=FGRCriterium(wanted_fgrtype=FGRType.LV)
        )
    ]
)

for lbk_code, fgr_code, bodem_code in [("HzHL", FGRType.DU, ["Zn21"]), ("Abcd", FGRType.LV, ["Abcd"]), (pd.NA, pd.NA, [pd.NA])]:
    row = pd.Series({"lbk": lbk_code, "fgr": fgr_code, "bodem": bodem_code})
    crit.check(row)
    print(str(crit))
    print(crit.evaluation)
    for sentence in crit.get_opm():
        print(sentence)
    print("--------")

((LBK is Herstellend hoogveen (C) of LBK is Hoogveenlandschap (T) of Bodem is Leemarme humuspodzolgronden (F)) en niet FGR is Laagveengebied (F))
TRUE
Dit is Hoogveenlandschap want LBK code HzHL.
FGR type is niet Laagveengebied, maar Duinen.
Dit is niet Leemarme humuspodzolgronden want bodemcode Zn21.
Dit is mogelijk Herstellend hoogveen want LBK code HzHL.
--------
((LBK is Herstellend hoogveen (F) of LBK is Hoogveenlandschap (C) of Bodem is Leemarme humuspodzolgronden (F)) en niet FGR is Laagveengebied (T))
FALSE
Dit is niet Herstellend hoogveen want LBK code Abcd.
Dit is mogelijk toch Hoogveenlandschap ondanks LBK code Abcd.
Dit is niet Leemarme humuspodzolgronden want bodemcode Abcd.
FGR type is Laagveengebied.
--------
((LBK is Herstellend hoogveen (C) of LBK is Hoogveenlandschap (C) of Bodem is Leemarme humuspodzolgronden (C)) en niet FGR is Laagveengebied (C))
CANNOT_BE_AUTOMATED
Dit vlak ligt niet mooi binnen één FGR-vlak.
Dit vlak ligt niet mooi binnen één bodemkaartvlak.
Dit 